In [1]:
from twython import Twython
import pandas as pd
import numpy as np
import datetime as dt
import time
import yaml
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re
import tqdm
from tqdm import tqdm

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [90]:
APP_KEY = 'OumzA854PAnWwA8VNjwZ6O1T1'
APP_SECRET = "aHWN7hVHFYGTJIqLG6d1sUgT2tzelmB7LNxtQjJz0RkmLDIFGW"

twitter = Twython(APP_KEY, APP_SECRET, oauth_version=2)
ACCESS_TOKEN = twitter.obtain_access_token()

In [91]:
twitter = Twython(APP_KEY, access_token=ACCESS_TOKEN)

In [5]:
keywords = ['music', 'jazz', 'rap', '(country music)', '(pop music)', '(rock music)', '(classical music)']
keywords = pd.DataFrame(keywords)
keywords.columns = ["Keywords"]
keywords.to_csv("Keywords.csv")
keywords = ['music', 'jazz', 'rap', '(country music)', '(pop music)', '(rock music)', '(classical music)']

In [ ]:
date = dt.datetime.now()
cnt = 0
tweetList = []
# for keyword in keywords:
tweets = twitter.cursor(twitter.search, q = '(classical music)' +' -filter:retweets', lang = 'en', count = 100, tweet_mode = "extended")
for item in tweets:
    if (cnt == 7000):
        cnt = 0
        tweetsDF = pd.DataFrame(tweetList)
        tweetsDF.to_csv("tweets_collected_"+str(date.day)+str(date.strftime("%m"))+"_"+keyword+".csv")
        tweetList = []
        break
    tweetList.append(item)
    cnt+=1

In [5]:
tweetsDF1 = pd.read_csv("Earlier collection/tweets_collected_1404_music.csv", dtype={'id':object})
tweetsDF1["keyword"] = "music"
tweetsDF2 = pd.read_csv("Earlier collection/tweets_collected_1404_jazz.csv", dtype={'id':object})
tweetsDF2["keyword"] = "jazz"
tweetsDF3 = pd.read_csv("Earlier collection/tweets_collected_1404_rap.csv", dtype={'id':object})
tweetsDF3["keyword"] = "rap"
tweetsDF4 = pd.read_csv("Earlier collection/tweets_collected_1404_(country music).csv", dtype={'id':object})
tweetsDF4["keyword"] = "country"
tweetsDF5 = pd.read_csv("Earlier collection/tweets_collected_1404_(pop music).csv", dtype={'id':object})
tweetsDF5["keyword"] = "pop"
tweetsDF6 = pd.read_csv("Earlier collection/tweets_collected_1404_(rock music).csv", dtype={'id':object})
tweetsDF6["keyword"] = "rock"
tweetsDF7 = pd.read_csv("Earlier collection/tweets_collected_1404_(classical music).csv", dtype={'id':object})
tweetsDF7["keyword"] = "classical"

In [6]:
tweetsDFList = [tweetsDF1,tweetsDF2,tweetsDF3,tweetsDF4,tweetsDF5,tweetsDF6,tweetsDF7]

In [9]:
date = dt.datetime.now()
finalTweetsDFList = []
for tweetsDF in tweetsDFList:
    cnt = 0
    tempList = []
    tweetList = []
    date = dt.datetime.now()
    for index, row in tweetsDF.iterrows():
        tempList.append(row["id"])
        if cnt%100 == 0:
            try:
                tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
                tempList = []
            except:
                print("Succesful till: "+cnt)
        cnt+=1
    tweetList.extend(twitter.lookup_status(id=tempList, tweet_mode = "extended"))
    keyword = tweetsDF["keyword"][0]
    tweetsDF = pd.DataFrame(tweetList)
    tweetsDF["keyword"] = keyword
    finalTweetsDFList.append(tweetsDF)
    tweetsDF.to_csv("Final Collection/tweets_recollected_"+str(date.day)+str(date.strftime("%m"))+"_"+keyword+".csv")

In [3]:
keywords = ['music', 'jazz', 'rap', 'country', 'pop', 'rock', 'classical']
finalTweetsDFList = []
for keyword in keywords:
    finalTweetsDFList.append(pd.read_csv("Final Collection/tweets_recollected_1704_"+keyword+".csv"))

In [21]:
for tweetsDF in finalTweetsDFList:
    if (tweetsDF.shape[0] != tweetsDF['id'].nunique()):
        print("Duplicate tweets found!")
print("Total number of unique tweets: ", pd.concat(tweetsDFList)['id'].nunique())

Duplicate tweets found!
Total number of unique tweets:  45210


In [22]:
stopWordsSet = set(stopwords.words('english')) 
stopWordsSet.add("u")
ids = []
for tweetsDF in tqdm(finalTweetsDFList):
    keyword = tweetsDF["keyword"][0]
    with open("Pre-Processed Tweets/tweets_text_"+keyword+".txt", "a+") as text_file:
        for index, row in tweetsDF.iterrows():
            if (row['id'] not in ids):
                ids.append(row['id'])
                tweetText = str.strip(re.sub('\s?https?://.*?[\s|\n]', '', row["full_text"]+" "))
                tweetText = str.strip(re.sub('@.*?[\s|\n]', '', tweetText+" "))
                tweetText = re.sub('&amp;', '', tweetText)
                tweetText = " ".join(re.findall("[a-zA-Z]+",tweetText))
                tokenizedText = word_tokenize(tweetText.lower())
                finalText = ''.join([WordNetLemmatizer().lemmatize(w) + " " for w in tokenizedText if not w in stopWordsSet and len(w) >= 3])
                text_file.write(str.strip(finalText) + "\n")
print(str(len(ids))+" unique tweets written to .txt")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [02:33<00:00, 21.86s/it]


44450 unique tweets written to .txt
